In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('HeartRateVariabilitySDNN.csv')

df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])
df = df.sort_values('startDate').reset_index(drop=True)


In [2]:
df.value.describe()

count    2993.000000
mean       28.527985
std        18.015771
min         4.115290
25%        17.327400
50%        24.115800
75%        33.707500
max       178.596000
Name: value, dtype: float64

In [24]:
# segment_duration_sec = 10
# start_time = df['startDate'].min()
# end_time = df['endDate'].max()

# current_start = start_time
# current_end = current_start + pd.Timedelta(seconds=segment_duration_sec)

# features_list = []

# while current_end <= end_time:
#     mask = (df['startDate'] < current_end) & (df['endDate'] > current_start)
#     segment = df.loc[mask]

#     if len(segment) >= 2:
#         values = segment['value'].values

#         hrv_sdnn = values.mean()

#         features_list.append({
#             'HRV_SDNN': hrv_sdnn,
#             'Segment_Start': current_start
#         })
#     current_start = current_end
#     current_end = current_start + pd.Timedelta(seconds=segment_duration_sec)

# feature_df = pd.DataFrame(features_list)

# print(feature_df.head())


In [2]:
df['HRV_SDNN'] = df['value']
df['HRV_SDNN_lag1'] = df['HRV_SDNN'].shift(1)
df['HRV_SDNN_lag2'] = df['HRV_SDNN'].shift(2)
cols = ['HRV_SDNN', 'HRV_SDNN_lag1', 'HRV_SDNN_lag2']

In [3]:
window_size = 5
X = df[cols]
X = X.fillna(0)
X_sequences = []
for i in range(len(X) - window_size):        
    X_seq = X[i:i+window_size]
    X_sequences.append(X_seq)
X_sequences = np.array(X_sequences)


In [5]:
from tensorflow.keras.models import load_model
model = load_model('/Users/tuoyuaner/Downloads/WESAD/lstm_emotion_model.h5')

model.save('lstm_emotion_model.keras')  

In [ ]:

y_pred_prob = model.predict(X_sequences)
y_pred = np.argmax(y_pred_prob, axis=1)

In [41]:
len(y_pred)

2988

In [44]:
pd.Series(y_pred).value_counts()

1    1821
0    1027
2     140
dtype: int64

In [ ]:
pip list

Package                       Version
----------------------------- -------------------
about-time                    3.1.1
absl-py                       1.0.0
alive-progress                2.4.1
appnope                       0.1.3
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
astor                         0.8.1
asttokens                     2.0.5
astunparse                    1.6.3
attrs                         20.3.0
autograd                      1.4
autograd-minimize             0.2.2
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.1
bert                          2.2.0
bert-tensorflow               1.0.4
bleach                        5.0.0
blis                          0.7.7
boto3                         1.23.9
botocore                      1.26.9
cached-property               1.5.2
cachetools                    5.0.0
catalogue                     2.0.7
certifi                       2021.10.8
cffi